In [9]:
import os

In [10]:
#!pip install -qU langchain-groq

In [11]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    groq_api_key = os.environ['GROQ_SECRET_KEY']

)

In [12]:
response = llm.invoke('The first person to land on moon was ?')
print(response.content)

The first person to walk on the Moon was Neil Armstrong, an American astronaut who stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the Moon.


In [13]:
#!pip install -q chromadb

In [14]:
import chromadb

client =  chromadb.Client()
collection = client.create_collection(name='my_collection')

collection.add(
    documents = [
        'This document about New York' ,
        "This document is about delhi"
    ] ,
    ids = ['id1','id2']
)


In [15]:
all_docs = collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This document about New York', 'This document is about delhi'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [16]:
doc = collection.get(ids=["id1"])
doc

{'ids': ['id1'],
 'embeddings': None,
 'documents': ['This document about New York'],
 'uris': None,
 'data': None,
 'metadatas': [None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [17]:
collection.query(
    query_texts = ['Query is about India'] ,
    n_results=2
)

{'ids': [['id2', 'id1']],
 'embeddings': None,
 'documents': [['This document is about delhi',
   'This document about New York']],
 'uris': None,
 'data': None,
 'metadatas': [[None, None]],
 'distances': [[0.7835131287574768, 1.640509843826294]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [18]:
collection.delete(ids= all_docs['ids'])
collection.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [19]:
collection.add(
    documents=[
        "this document is about New Delhi" ,
        "This document is about Delhi"
    ],
    ids = ['id3','id4'] ,
    metadatas = [
        {"url" : "https://en.wikipedia.org/wiki/New_York_City"} ,
        {"url" : "https://en.wikipedia.org/wiki/New_Delhi"}
    ]
)

In [20]:
collection.query(
    query_texts = ['Query is about India'] ,
    n_results=2
)

{'ids': [['id4', 'id3']],
 'embeddings': None,
 'documents': [['This document is about Delhi',
   'this document is about New Delhi']],
 'uris': None,
 'data': None,
 'metadatas': [[{'url': 'https://en.wikipedia.org/wiki/New_Delhi'},
   {'url': 'https://en.wikipedia.org/wiki/New_York_City'}]],
 'distances': [[0.7835131287574768, 0.8420597314834595]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [21]:
#! pip install -q langchain-community bs4

In [39]:
from langchain_community.document_loaders import WebBaseLoader

#link = input('Link to create mail for :')

loader = WebBaseLoader('https://jobs.nike.com/job/R-42328?from=job%20search%20funnel')

page_data = loader.load().pop().page_content

print(page_data)

Apply for Retail Associate, SEAS - Las Vegas South

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsRetail StoresRetail Associate, SEAS - Las Vegas SouthLas Vegas, NevadaBecome part of t

In [40]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [56]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

json_res = json_parser.parse(res.content)[0]
json_res

{'role': 'Retail Associate, SEAS - Las Vegas South',
 'experience': 'Seasonal – 0-40 hours per week, including nights and weekends',
 'skills': 'Must be at least 18 (U.S), Flexibility to work nights, weekends and holidays based on store needs, You want to learn and understand footwear, apparel and accessories and pass that knowledge onto your customers, Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time *with or without assistance',
 'description': 'Be an All Star as a Converse Retail Associate\nYou’re energetic, a rebel – a game-changer – and we want you on our team. Bring your authentic form of self-expression together with other creatives, creating a unique vibe that is unlike anywhere else. Our shoes are iconic, just like you and just like our customers. Be part of continuing our legacy of inspiring, motivating and embracing the world of sport, culture, and creativity.\nAt Converse, we celebrate and welcome you for

In [58]:
type(res)

langchain_core.messages.ai.AIMessage

In [59]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [60]:
import uuid
import chromadb

client =  chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count() :
    for _ , row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas = {
                           "links" : row["Links"]
                       },
                       ids = [str(uuid.uuid4())])

In [61]:
job = json_res

In [62]:
job

{'role': 'Retail Associate, SEAS - Las Vegas South',
 'experience': 'Seasonal – 0-40 hours per week, including nights and weekends',
 'skills': 'Must be at least 18 (U.S), Flexibility to work nights, weekends and holidays based on store needs, You want to learn and understand footwear, apparel and accessories and pass that knowledge onto your customers, Physical requirements include the ability to bend, squat, reach, climb a ladder and stand for extended periods of time *with or without assistance',
 'description': 'Be an All Star as a Converse Retail Associate\nYou’re energetic, a rebel – a game-changer – and we want you on our team. Bring your authentic form of self-expression together with other creatives, creating a unique vibe that is unlike anywhere else. Our shoes are iconic, just like you and just like our customers. Be part of continuing our legacy of inspiring, motivating and embracing the world of sport, culture, and creativity.\nAt Converse, we celebrate and welcome you for

In [63]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/ios-portfolio'}]]

In [64]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Your Retail Potential with AtliQ's AI Solutions

Dear [Hiring Manager's Name],

I came across the job posting for a Retail Associate at Converse, and I was impressed by the company's commitment to celebrating individuality and creativity. As a Business Development Executive at AtliQ, I'd like to introduce you to our AI-powered solutions that can help you streamline your retail operations and enhance the customer experience.

AtliQ has a proven track record of empowering businesses like yours with tailored solutions that drive scalability, process optimization, and cost reduction. Our expertise in AI and software consulting enables us to identify areas of improvement and implement customized tools to address specific challenges.

I'd like to highlight a few examples from our portfolio that demonstrate our capabilities:

* [Link: https://example.com/python-portfolio] - Our Python-based solution for inventory management helped a leading fashion brand reduce stockroom error